In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.metrics import r2_score
import pandas as pd

SyntaxError: invalid syntax (182603721.py, line 3)

In [ ]:
data = pd.read_csv('scaled_data.csv')
data = data[:100]
X = data.drop('popularity', axis=1)
y = data['popularity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
ridge_cv = RidgeCV(alphas=[0.1, 1.0, 10.0], cv=5)
ridge_cv.fit(X_train_scaled, y_train)

In [ ]:
y_pred = ridge_cv.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
print(f'R^2 Score: {r2}')